In [5]:
import pandas as pd
import os

In [6]:
input_dir  = "."
output_dir = "./3K_New_Analysis"

In [7]:
columns_to_copy = [
    "Number of all Red Total LDS in inclusion",
    "Number of all Red Total LDS not in inclusion",
    "Number of Red-only Total LDS in inclusion",
    "Number of Red-only Total LDS not in inclusion",
]

files_to_process = [
    "12225_SUMMARY.xlsx",
    "012925_SUMMARY.xlsx",
    "020525_SUMMARY.xlsx",
    "021225_SUMMARY.xlsx",
    "021925_SUMMARY.xlsx"
]

In [8]:
def safe_div(numer, denom):
    """Return numer/denom, or 0 if denom is zero or invalid."""
    try:
        return numer / denom if denom and denom != 0 else 0
    except Exception:
        return 0

# %%
# Build list of candidate Excel filenames:
all_excels = [
    f for f in os.listdir(input_dir)
    if f.lower().endswith((".xls", ".xlsx"))
]

# Apply whitelist if provided:
if files_to_process:
    to_loop = [f for f in all_excels if f in files_to_process]
else:
    to_loop = all_excels

# Process each file
for fname in to_loop:
    in_path = os.path.join(input_dir, fname)
    print(f"→ Reading {fname}…")
    df = pd.read_excel(in_path)

    # Identify which of our key columns are present
    present = [c for c in columns_to_copy if c in df.columns]
    missing = set(columns_to_copy) - set(present)
    if missing:
        print(f"⚠️  Warning: {fname} is missing columns {missing}")

    # Build the new DataFrame
    new_df = df[present].copy()

    # Insert Filename pulled from the sheet (or fallback to the file’s name)
    if "Filename" in df.columns:
        new_df.insert(0, "Filename", df["Filename"])
    else:
        new_df.insert(0, "Filename", fname)

    # Compute the two ratio columns
    new_df["Red-only/All Red Total (in inclusion)"] = new_df.apply(
        lambda r: safe_div(
            r.get("Number of Red-only Total LDS in inclusion", 0),
            r.get("Number of all Red Total LDS in inclusion", 0),
        ),
        axis=1
    )
    new_df["Red-only/All Red Total (outside inclusion)"] = new_df.apply(
        lambda r: safe_div(
            r.get("Number of Red-only Total LDS not in inclusion", 0),
            r.get("Number of all Red Total LDS not in inclusion", 0),
        ),
        axis=1
    )

    # Write out the processed file
    out_name = os.path.splitext(fname)[0] + "_important_columns.xlsx"
    out_path = os.path.join(output_dir, out_name)
    new_df.to_excel(out_path, index=False)
    print(f"✅  Wrote {out_name}")

# %%
print("🎉 All files processed.")

→ Reading 012925_SUMMARY.xlsx…
✅  Wrote 012925_SUMMARY_important_columns.xlsx
→ Reading 020525_SUMMARY.xlsx…
✅  Wrote 020525_SUMMARY_important_columns.xlsx
→ Reading 021225_SUMMARY.xlsx…
✅  Wrote 021225_SUMMARY_important_columns.xlsx
→ Reading 021925_SUMMARY.xlsx…
✅  Wrote 021925_SUMMARY_important_columns.xlsx
→ Reading 12225_SUMMARY.xlsx…
✅  Wrote 12225_SUMMARY_important_columns.xlsx
🎉 All files processed.
